In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'

solver = "ECOS"
save_data = False

from kelly_gambling import *
dataset = "kelly"

from osmm import OSMM
import pickle
import datetime

device = cpu


In [2]:
max_num_rounds = 100

In [3]:
alg_props = {    
    "r=0, M=1": [0, 0, 1],
    "r=0, M=20": [1, 0, 20],
    "r=0, M=50": [2, 0, 50],
}
    
num_algs = len(alg_props)

In [10]:
repeat_times = 3

In [11]:
# Define structs:
objfs = np.zeros((repeat_times, num_algs, max_num_rounds))
objfs_validation = np.zeros((repeat_times, num_algs, max_num_rounds))
runtimes = np.zeros((repeat_times, num_algs, max_num_rounds))
iters_takens = np.zeros((repeat_times, num_algs))
damping_facs = np.ones((repeat_times, num_algs, max_num_rounds))
mus = np.ones((repeat_times, num_algs, max_num_rounds))
ts = np.ones((repeat_times, num_algs, max_num_rounds))
Xs = np.zeros((repeat_times, n, num_algs, max_num_rounds))
lower_bounds = np.ones((repeat_times, num_algs, max_num_rounds)) * (-np.inf)
best_xs = np.zeros((repeat_times, n, num_algs))
v_norms = np.zeros((repeat_times, num_algs, max_num_rounds))
num_f_evals = np.zeros((repeat_times, num_algs, max_num_rounds))
rms_residuals = np.zeros((repeat_times, num_algs, max_num_rounds))
q_norms = np.zeros((repeat_times, num_algs, max_num_rounds))
f_grad_norms = np.zeros((repeat_times, num_algs, max_num_rounds))
time_cost_details = np.zeros((repeat_times, 5, num_algs, max_num_rounds))
baseline_opt_objf = []
baseline_time_cost = []

In [12]:
osmm_prob = OSMM()
osmm_prob.f_torch.function = my_f_torch
osmm_prob.g_cvxpy.variable = g_var
osmm_prob.g_cvxpy.objective = g_obj
osmm_prob.g_cvxpy.constraints = g_constr

In [13]:
init_val = np.ones(n) / n

In [14]:
for repeat_idx in range(repeat_times):
    print("repeat =", repeat_idx)
    print("dataset = %s:" % (dataset))
    W = generate_random_data(None)
    print("End generate data")
    osmm_prob.f_torch.W_torch = torch.tensor(W, dtype=torch.float)
    for alg_name in alg_props:
        print("    " + alg_name + ":")
        alg_idx = alg_props[alg_name][0]
        H_rank = alg_props[alg_name][1]
        M = alg_props[alg_name][2]
        
        osmm_prob.solve(init_val, max_iter=max_num_rounds, 
                        hessian_rank=H_rank, gradient_memory=M,
                        solver=solver, use_termination_criteria=False, verbose=True, use_Hutchinson_init=False)

        Xs[repeat_idx, :, alg_idx, :] = osmm_prob.method_results["var_iters"]
        objfs[repeat_idx, alg_idx, :] = osmm_prob.method_results["objf_iters"]
        objfs_validation[repeat_idx, alg_idx, :] = osmm_prob.method_results["objf_validate_iters"]
        lower_bounds[repeat_idx, alg_idx, :] = osmm_prob.method_results["lower_bound_iters"]
        damping_facs[repeat_idx, alg_idx, :] = osmm_prob.method_results["lam_iters"]
        mus[repeat_idx, alg_idx, :] = osmm_prob.method_results["mu_iters"]
        ts[repeat_idx, alg_idx, :] = osmm_prob.method_results["t_iters"]
        runtimes[repeat_idx, alg_idx, :] = osmm_prob.method_results["time_iters"]
        best_xs[repeat_idx, :, alg_idx] = osmm_prob.method_results["soln"]
        v_norms[repeat_idx, alg_idx, :] = osmm_prob.method_results["v_norm_iters"]
        num_f_evals[repeat_idx, alg_idx, :] = osmm_prob.method_results["num_f_evals_iters"]
        rms_residuals[repeat_idx, alg_idx, :] = osmm_prob.method_results["rms_res_iters"]
        f_grad_norms[repeat_idx, alg_idx, :] = osmm_prob.method_results["f_grad_norm_iters"]
        q_norms[repeat_idx, alg_idx, :] = osmm_prob.method_results["q_norm_iters"]
        time_cost_details[repeat_idx, :, alg_idx, :] = osmm_prob.method_results["time_detail_iters"]
        iters_takens[repeat_idx, alg_idx] = osmm_prob.method_results["total_iters"]
        print("")

repeat = 0
dataset = kelly:
End generate data
    r=0, M=1:
iter = 10, objf = -2.486e-02, lower bound = -1.419e-01, RMS residual = 1.019e-02, ||G||_F = 0.000e+00
iter = 20, objf = -4.459e-02, lower bound = -1.291e-01, RMS residual = 1.247e-02, ||G||_F = 0.000e+00
iter = 30, objf = -5.964e-02, lower bound = -9.135e-02, RMS residual = 6.282e-03, ||G||_F = 0.000e+00
iter = 40, objf = -6.354e-02, lower bound = -7.409e-02, RMS residual = 3.222e-03, ||G||_F = 0.000e+00
iter = 50, objf = -6.412e-02, lower bound = -6.884e-02, RMS residual = 1.413e-03, ||G||_F = 0.000e+00
iter = 60, objf = -6.424e-02, lower bound = -6.469e-02, RMS residual = 7.441e-04, ||G||_F = 0.000e+00
iter = 70, objf = -6.426e-02, lower bound = -6.430e-02, RMS residual = 5.510e-06, ||G||_F = 0.000e+00
iter = 80, objf = -6.426e-02, lower bound = -6.426e-02, RMS residual = 4.866e-07, ||G||_F = 0.000e+00
iter = 90, objf = -6.426e-02, lower bound = -6.426e-02, RMS residual = 4.622e-07, ||G||_F = 0.000e+00
      Terminated. Num 

/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/problems/problem.py:1279: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


iter = 70, objf = -6.426e-02, lower bound = -6.426e-02, RMS residual = 4.353e-06, ||G||_F = 0.000e+00
iter = 80, objf = -6.426e-02, lower bound = -6.426e-02, RMS residual = 1.635e-06, ||G||_F = 0.000e+00
iter = 90, objf = -6.426e-02, lower bound = -6.426e-02, RMS residual = 1.825e-06, ||G||_F = 0.000e+00
      Terminated. Num iterations = 99, objf = -6.426e-02, lower bound = -6.426e-02, RMS residual = 2.156e-06.
      Time elapsed (secs): 3.659802.


repeat = 1
dataset = kelly:
End generate data
    r=0, M=1:
iter = 10, objf = -2.596e-02, lower bound = -1.503e-01, RMS residual = 1.445e-02, ||G||_F = 0.000e+00
iter = 20, objf = -4.635e-02, lower bound = -1.138e-01, RMS residual = 1.040e-02, ||G||_F = 0.000e+00
iter = 30, objf = -5.983e-02, lower bound = -8.785e-02, RMS residual = 6.265e-03, ||G||_F = 0.000e+00
iter = 40, objf = -6.268e-02, lower bound = -7.186e-02, RMS residual = 3.204e-03, ||G||_F = 0.000e+00
iter = 50, objf = -6.321e-02, lower bound = -6.802e-02, RMS residual = 5.971e

In [15]:
print("W shape =", W.shape)
_, N = W.shape
print("log10 N =", np.log10(N), ", n =", n)

W shape = (201, 1000)
log10 N = 3.0 , n = 200


In [11]:
stuff = {}
stuff["objfs"] = objfs
stuff["objfs_validation"] = objfs_validation
stuff["lower_bounds"] = lower_bounds
stuff["runtimes"] = runtimes
stuff["rms_residuals"] = rms_residuals
stuff["f_grad_norms"] = f_grad_norms
stuff["q_norms"] = q_norms
stuff["damping_facs"] = damping_facs
stuff["ts"] = ts
stuff["Xs"] = Xs
stuff["best_xs"] = best_xs
stuff["v_norms"] = v_norms
stuff["num_f_evals"] = num_f_evals
stuff["iters_takens"] = iters_takens
stuff["time_cost_details"] = time_cost_details
stuff["alg_props"] = list(alg_props.keys())

stuff["baseline_opt_objf"] = baseline_opt_objf
stuff["baseline_time_cost"] = baseline_time_cost
now = datetime.datetime.now()
mmddyyhhmm = ("%d_%d_%d_%d_%d" % (now.month, now.day, now.year, now.hour, now.minute))
part_of_out_fn = dataset + "" + mmddyyhhmm

In [12]:
if save_data:
    pickle.dump(stuff, open("ECOS_data_%s.pkl" % (part_of_out_fn), "wb"))

## timing results

In [21]:
ep = 1e-6
solve_time_all = np.zeros((repeat_times, num_algs))
solve_iter_all = np.zeros((repeat_times, num_algs))
f_eval_all = np.zeros((repeat_times, num_algs))
for repeat_idx in range(repeat_times):
    for alg_idx in range(len(alg_props)):
        subopt = objfs[repeat_idx, alg_idx, :] - np.min(objfs[repeat_idx, :, :])
        iter_reach_ep = max_num_rounds
        for i in range(max_num_rounds):
            if subopt[i] <= ep:
                iter_reach_ep = i
                break
        f_evals = np.mean(num_f_evals[repeat_idx, alg_idx, 1:iter_reach_ep], axis=0)
#         print("iters to reach 1e-6 subopt = ", iter_reach_ep)
#         print("solve time to reach 1e-6 subopt = ", np.sum(runtimes[repeat_idx, alg_idx, 0:iter_reach_ep + 1]))
#         print("f evals", f_evals)
        solve_time_all[repeat_idx, alg_idx] = np.sum(runtimes[repeat_idx, alg_idx, 0:iter_reach_ep + 1])
        solve_iter_all[repeat_idx, alg_idx] = iter_reach_ep
        f_eval_all[repeat_idx, alg_idx] = f_evals

In [22]:
print(np.mean(solve_time_all[:, :], axis=0))
print(np.mean(solve_iter_all[:, :], axis=0))
print(np.mean(f_eval_all[:, :], axis=0))

[1.36160723 1.33172282 1.49965858]
[64.33333333 51.33333333 49.66666667]
[3.99666922 3.47150183 3.38407029]


In [23]:
# mean_time_details = np.mean(time_cost_details, axis=3)
print("time cost detail")
print("===============f======grad f====subp====L_k====")
for alg_idx in range(len(alg_props)):
    f_eval = np.mean(time_cost_details[:, 0, alg_idx, 1::])
    f_grad_eval = np.mean(time_cost_details[:, 1, alg_idx, 1::])
    subp_ave = np.mean(time_cost_details[:, 2, alg_idx, 2::])
    subp_first = time_cost_details[:, 2, alg_idx, 1]
    L_k_ave = np.mean(time_cost_details[:, 3, alg_idx, 20:100:10])
    L_k_first = time_cost_details[:, 3, alg_idx, 10]
    print_results = [f_eval, f_grad_eval, subp_ave, L_k_ave]
    print(alg_idx, ['{:g}'.format(float('{:.2g}'.format(print_results[i]))) for i in range(4)])

time cost detail
===============f======grad f====subp====L_k====
0 ['0.00036', '0.00037', '0.018', '0.0075']
1 ['0.0006', '0.00038', '0.024', '0.012']
2 ['0.00061', '0.00038', '0.031', '0.023']
